In [1]:
import pandas as pd

from lib.reddit_api import Reddit_API

In [2]:
SEARCH_TERMS = {
    'alcoholic_beverages':
    ["alcohol","drinks","cocktails","beer","wine","liquor","spirits",
    "happyhour","mixology","bartender","craftbeer","winelover","drinkstagram",
    "whiskey","vodka","gin","rum","tequila","drinking","drinkresponsibly"]
}


In [3]:
to_fp="./creds/monkey"
reddit = Reddit_API()
pickle_name = 'monkey'
reddit.connect_reddit(pickle_name = to_fp)
raw_reddit_posts = pd.DataFrame()
for topic, search_terms in SEARCH_TERMS.items():
    for search_term in search_terms:
            print(f'searching {search_term}...')
            results_df = reddit.post_search(search_term)
            results_df['search_term'] = search_term
            results_df['topic'] = topic
            raw_reddit_posts = pd.concat([raw_reddit_posts, results_df])
            raw_reddit_posts = raw_reddit_posts[~raw_reddit_posts.duplicated()].reset_index(drop = True)
raw_reddit_posts['created'] = pd.to_datetime(raw_reddit_posts['created_utc'], unit='s', utc=True)
# Convert the UTC timestamps to local time
local_timezone = 'America/Chicago'  # Replace with the desired timezone
raw_reddit_posts['created'] = raw_reddit_posts['created'].dt.tz_convert(local_timezone)#.dt.strftime('%Y_%b_%d %H_%M')
print(f'''
raw_reddit_posts.shape: {raw_reddit_posts.shape}
''')
print(raw_reddit_posts.head())


<Response [200]>
searching alcohol...
searching drinks...
searching cocktails...
searching beer...
searching wine...
searching liquor...
searching spirits...
searching happyhour...
searching mixology...
searching bartender...
searching craftbeer...
searching winelover...
searching drinkstagram...
searching whiskey...
searching vodka...
searching gin...
searching rum...
searching tequila...
searching drinking...
searching drinkresponsibly...

raw_reddit_posts.shape: (4057, 16)

                    created   created_utc          subreddit subreddit_id  \
0 2023-08-02 19:49:02-05:00  1.691024e+09          AskReddit     t5_2qh1i   
1 2023-07-13 03:43:57-05:00  1.689238e+09  NoStupidQuestions     t5_2w844   
2 2023-06-28 07:19:25-05:00  1.687955e+09     TooAfraidToAsk     t5_2ssp7   
3 2023-03-02 15:30:39-06:00  1.677793e+09             loseit     t5_2rz8w   
4 2023-05-23 09:16:55-05:00  1.684851e+09            shrooms     t5_2qszq   

                                            selftext au

In [4]:
raw_reddit_posts.head()

,created,created_utc,subreddit,subreddit_id,selftext,author_fullname,title,name,id,author,num_comments,url,upvote_ratio,ups,search_term,topic
0,2023-08-02 19:49:02-05:00,1.691024e+09,AskReddit,t5_2qh1i,,t2_wb9ut,"People who don't drink alcohol, why?",t3_15gpdyb,15gpdyb,johnnylgarfield,32587,https://www.reddit.com/r/AskReddit/comments/15...,0.72,16221,alcohol,alcoholic_beverages
1,2023-07-13 03:43:57-05:00,1.689238e+09,NoStupidQuestions,t5_2w844,"I’m 15 and try as I might, I just can’t seem t...",t2_afca0qz7,Why are adults so hooked on alcohol?,t3_14yf8vr,14yf8vr,SalttySkies,4211,https://www.reddit.com/r/NoStupidQuestions/com...,0.80,6936,alcohol,alcoholic_beverages
2,2023-06-28 07:19:25-05:00,1.687955e+09,TooAfraidToAsk,t5_2ssp7,I'm trying to stop drinking and it feels so la...,t2_3joiof8k,"Do people really enjoy life without alcohol, o...",t3_14l80bn,14l80bn,TA2556,2206,https://www.reddit.com/r/TooAfraidToAsk/commen...,0.91,4713,alcohol,alcoholic_beverages
3,2023-03-02 15:30:39-06:00,1.677793e+09,loseit,t5_2rz8w,I'm not saying you can't lose weight while dri...,t2_1uc2ctml,"If you're serious about this, take a break fro...",t3_11geo7u,11geo7u,noodlesworldwide,285,https://www.reddit.com/r/loseit/comments/11geo...,0.96,2538,alcohol,alcoholic_beverages
4,2023-05-23 09:16:55-05:00,1.684851e+09,shrooms,t5_2qszq,I keep reading about how mixing alcohol and sh...,t2_117e6h,Shrooms and alcohol,t3_13ppuwz,13ppuwz,Cristhekid,364,https://www.reddit.com/r/shrooms/comments/13pp...,0.93,227,alcohol,alcoholic_beverages


In [5]:
raw_reddit_posts['created'] = pd.to_datetime(raw_reddit_posts['created']).dt.strftime("%Y-%b-%d %H:%M")

In [6]:
raw_reddit_posts['created'].iloc[0]

'2023-Aug-02 19:49'

In [7]:
raw_reddit_cols = ['id','created','subreddit','subreddit_id','selftext','author_fullname','title',
                        'name','url']
raw_reddit_pulls = raw_reddit_posts[raw_reddit_cols]

In [8]:
def get_posts():
    reddit = Reddit_API()

    to_fp='./monkey.pkl'
    pickle_name = 'monkey'
    get_blob(CONFIG['bucket_name'], CONFIG['reddit_api_fp'], to_fp)
    print(os.listdir())


    reddit.connect_reddit(pickle_name = pickle_name)
    raw_reddit_posts = pd.DataFrame()
    for topic, search_terms in SEARCH_TERMS.items():
        for search_term in search_terms:
            print(f'searching {search_term}...')
            results_df = reddit.post_search(search_term)
            results_df['search_term'] = search_term
            results_df['topic'] = topic
            raw_reddit_posts = pd.concat([raw_reddit_posts, results_df])
            raw_reddit_posts = raw_reddit_posts[~raw_reddit_posts.duplicated()].reset_index(drop = True)
    raw_reddit_posts['created'] = pd.to_datetime(raw_reddit_posts['created_utc'], unit='s', utc=True)
    # Convert the UTC timestamps to local time
    local_timezone = 'America/Chicago'  # Replace with the desired timezone
    raw_reddit_posts['created'] = raw_reddit_posts['created'].dt.tz_convert(local_timezone)#.dt.strftime('%Y_%b_%d %H_%M')
    print(f'''
    raw_reddit_posts.shape: {raw_reddit_posts.shape}
    ''')
    print(raw_reddit_posts.head())
    return raw_reddit_posts

In [9]:
CREDIENTIALS_PATH = './lib//trusty-sentinel-283421-8d4e435c814a.json'

In [28]:
!pip install google-api-python-client
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud-storage

  Using cached google_cloud_bigquery-3.11.4-py2.py3-none-any.whl (219 kB)
  Using cached proto_plus-1.22.3-py3-none-any.whl (48 kB)
  Using cached grpcio-1.57.0.tar.gz (24.7 MB)
  Preparing metadata (setup.py) ... done
  Using cached grpcio_status-1.57.0-py3-none-any.whl (5.1 kB)
  Created wheel for grpcio: filename=grpcio-1.57.0-cp310-cp310-macosx_10_10_x86_64.whl size=4259904 sha256=29dc5cbe4bc44984d1165c3b91cdde85e1209ff6ec50d52fa75bc8ed40ac4802
  Stored in directory: /Users/stevenhansen/Library/Caches/pip/wheels/6d/fb/80/c244d0eb3892d472c21f9636a6dbf79fb69a2109e52f33ec2c
Successfully built grpcio


In [10]:
from google.cloud import bigquery
from google.oauth2 import service_account
def get_big_query(query, credientials_path = './plugins/oceanic-variety-386313-5611c0e63164.json', project_id = 'oceanic-variety-386313'):
    
    to_fp = CREDIENTIALS_PATH
    #get_blob(CONFIG['bucket_name'], CONFIG['bg_api_fp'], to_fp)
    credentials = service_account.Credentials.from_service_account_file(to_fp)
    client = bigquery.Client(credentials= credentials,project=project_id)
    print(f'executing BigQuery: {query}')
    query_job = client.query(query)
    results = query_job.result()
    return results.to_dataframe()

In [11]:
import os
os.listdir('./lib')

['reddit_api.py',
 'bucket_access.py',
 'trusty-sentinel-283421-8d4e435c814a.json',
 '.DS_Store',
 '__init__.py',
 '__pycache__',
 'api_connector.py',
 '.ipynb_checkpoints',
 'big_query_access.py',
 'reddit_pulls.py',
 'oceanic-variety-386313-5611c0e63164.json']

In [12]:
#specify number of rows in each chunk
n=100

#split DataFrame into chunks
list_df = [raw_reddit_pulls[i:i+n] for i in range(0,len(raw_reddit_pulls),n)]

In [13]:
list_df[0]

,id,created,subreddit,subreddit_id,selftext,author_fullname,title,name,url
0,15gpdyb,2023-Aug-02 19:49,AskReddit,t5_2qh1i,,t2_wb9ut,"People who don't drink alcohol, why?",t3_15gpdyb,https://www.reddit.com/r/AskReddit/comments/15...
1,14yf8vr,2023-Jul-13 03:43,NoStupidQuestions,t5_2w844,"I’m 15 and try as I might, I just can’t seem t...",t2_afca0qz7,Why are adults so hooked on alcohol?,t3_14yf8vr,https://www.reddit.com/r/NoStupidQuestions/com...
2,14l80bn,2023-Jun-28 07:19,TooAfraidToAsk,t5_2ssp7,I'm trying to stop drinking and it feels so la...,t2_3joiof8k,"Do people really enjoy life without alcohol, o...",t3_14l80bn,https://www.reddit.com/r/TooAfraidToAsk/commen...
3,11geo7u,2023-Mar-02 15:30,loseit,t5_2rz8w,I'm not saying you can't lose weight while dri...,t2_1uc2ctml,"If you're serious about this, take a break fro...",t3_11geo7u,https://www.reddit.com/r/loseit/comments/11geo...
4,13ppuwz,2023-May-23 09:16,shrooms,t5_2qszq,I keep reading about how mixing alcohol and sh...,t2_117e6h,Shrooms and alcohol,t3_13ppuwz,https://www.reddit.com/r/shrooms/comments/13pp...
...,...,...,...,...,...,...,...,...,...
95,13joa9u,2023-May-16 21:06,ask,t5_2qhlj,Since it’s been proven medically that there is...,t2_hp8xbg48,Why has alcohol became so socially acceptable ...,t3_13joa9u,https://www.reddit.com/r/ask/comments/13joa9u/...
96,12u9kkh,2023-Apr-21 11:18,explainlikeimfive,t5_2sokd,,t2_wwzpg,"ELI5: Why is ""older"" whiskey generally conside...",t3_12u9kkh,https://www.reddit.com/r/explainlikeimfive/com...
97,z8q6at,2022-Nov-30 06:24,Damnthatsinteresting,t5_2xxyj,,t2_22doecft,"Three year-old Robert Quigley smoking a cigar,...",t3_z8q6at,https://i.redd.it/3b62p8hkh43a1.jpg
98,12qk2sy,2023-Apr-18 06:29,explainlikeimfive,t5_2sokd,,t2_3q6xi58z,"ELI5: If we use alcohol as disinfectant, why d...",t3_12qk2sy,https://www.reddit.com/r/explainlikeimfive/com...


In [14]:
table_name = 'reddit_db.raw_reddit_pulls'

for df in list_df:
    values = str([tuple(value) for value in df.values]).replace('[','').replace(']','')

    query = f"""
    INSERT INTO {table_name} {str(tuple(df.columns)).replace("'","")}
    VALUES {values}
    """
    get_big_query(query, credientials_path = CREDIENTIALS_PATH, project_id = 'trusty-sentinel-283421')

executing BigQuery: 
    INSERT INTO reddit_db.raw_reddit_pulls (id, created, subreddit, subreddit_id, selftext, author_fullname, title, name, url)
    VALUES ('15gpdyb', '2023-Aug-02 19:49', 'AskReddit', 't5_2qh1i', '', 't2_wb9ut', "People who don't drink alcohol, why?", 't3_15gpdyb', 'https://www.reddit.com/r/AskReddit/comments/15gpdyb/people_who_dont_drink_alcohol_why/'), ('14yf8vr', '2023-Jul-13 03:43', 'NoStupidQuestions', 't5_2w844', 'I’m 15 and try as I might, I just can’t seem to wrap my head around the infatuation adults have with alcohol. I understand that most things, when properly moderated, can be consumed without much issue but alcohol just seems to be a toxic relationship that always ends up creating problems for everyone. Whether it be drunk texts to your exes, DUI’s, or just general asshole-ery, people always run back to it. \nI’ve heard stories of people being hesitant to attend weddings because of the dry factor. I’ve even heard about people taking shots or relying o

In [80]:
!pip install 'google-cloud-bigquery[pandas]'